# Dynamic risk scores practical session

In this lab you'll practice:
* (static) survival analysis
  * basic data preprocessing: 1 hot encoding, mean imputation, normalisation.
  * boosting model: CoxBoost
  * survival random forests
* dynamic survival analysis
  * basic data processing
  * 2 stage model
  * CNN


# Prepare the environment

install the necessary packages in the current environment

In [ ]:
!pip install pandas numpy matplotlib lifelines sklearn scikit-survival statsmodels tensorflow keras

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# (Static) Risk score

In the first part of our lab we'll use an open source dataset to evaluate some of the models used for Survival Analysis. We will use Concordance Index on test to score them. We will prepare the data, split the data in train and test, tune the hyperparameters using 5 fold cross validation on the train set, and evaluate each model on the test set. 

## The dataset

We'll use the lung cancer dataset (from the R `survival` package, see https://stat.ethz.ch/R-manual/R-devel/library/survival/html/lung.html)

* `inst`: Institution code
* `time`: Survival time in days
* `status`: censoring status 1=censored, 2=dead
* `age`: Age in years
* `sex`: Male=1 Female=2
* `ph.ecog`: ECOG performance score (0=good 5=dead)
* `ph.karno`: Karnofsky performance score as rated by physician
* `pat.karno`: Karnofsky performance score as rated by patient
* `meal.cal`: Calories consumed at meals
* `wt.loss`: Weight loss in last six months


In [ ]:
cancer = pd.read_csv("data/cancer.csv", index_col=0)
cancer.head()

In [ ]:
categorical_variables = ["inst", "ph.ecog"]
continuous_variables = set(cancer.columns) - set(["time", "status", "sex"]) - set(categorical_variables)

print(f"the continuous variables are {continuous_variables}")
print(f"the categorical variables are {categorical_variables}")

print(f"the inst column only has {len(cancer.inst.unique())} unique values")
inst_freqs = cancer.groupby('inst').size()
print(f"{np.sum(inst_freqs < 10)} values have less than 10 instances in the dataset, we'll make a special institution for them")
print(f"we will make a special institution for {list(inst_freqs.index[inst_freqs < 10])}, together with missing values")
print("institution will also be 1 hot encoded")


The `inst` column tells us from which hospital this subjects comes, it might have an impact on survival probability, so we keep this column. However, as shown above for some institutions we don't have enough data, and this will likely lead to problems. One possible approach is to remove the institutions that don't have enough subjects in this dataset. They will all be handled as if they were the same institution. This can be done quickly by simply assigning NaN to those subjects (we will handle NaNs later in this script). 

Now find all the rows with `inst` equal to a value that has less than 10 instances in the dataset and set `inst` to NaN for that rows. 

In [ ]:
# set `cancer.inst` to np.nan if that institution has less than 10 samples in this dataset
# in other words if is in this list list(inst_freqs.index[inst_freqs < 10])



## 1 hot encoding 

Let's now 1 hot encode the columns that are in the `categorical_variables` variable, you can use `sklearn.preprocessing.OneHotEncoder` for this task.

To avoid colinearity make sure you drop one of the output columns (e.g. using `drop="first"` in `OneHotEncoder`).

After this step the dataset should look similar to this (not necessarily with the same column names):

| time | status | age | sex | ph.karno | pat.karno | meal.cal | wt.loss | inst_3.0 | inst_6.0 | ... | inst_12.0 | inst_13.0 | inst_16.0 | inst_21.0 | inst_22.0 | inst_nan | ph.ecog_1.0 | ph.ecog_2.0 | ph.ecog_3.0 | ph.ecog_nan |     |
|-----:|-------:|----:|----:|---------:|----------:|---------:|--------:|---------:|---------:|----:|----------:|----------:|----------:|----------:|----------:|---------:|------------:|------------:|------------:|------------:|-----|
|    1 |    306 |   2 |  74 |        1 |      90.0 |    100.0 |  1175.0 |      NaN |      1.0 | 0.0 |       ... |       0.0 |       0.0 |       0.0 |       0.0 |      0.0 |         0.0 |         1.0 |         0.0 |         0.0 | 0.0 |
|    2 |    455 |   2 |  68 |        1 |      90.0 |     90.0 |  1225.0 |     15.0 |      1.0 | 0.0 |       ... |       0.0 |       0.0 |       0.0 |       0.0 |      0.0 |         0.0 |         0.0 |         0.0 |         0.0 | 0.0 |
|    3 |   1010 |   1 |  56 |        1 |      90.0 |     90.0 |     NaN |     15.0 |      1.0 | 0.0 |       ... |       0.0 |       0.0 |       0.0 |       0.0 |      0.0 |         0.0 |         0.0 |         0.0 |         0.0 | 0.0 |
|    4 |    210 |   2 |  57 |        1 |      90.0 |     60.0 |  1150.0 |     11.0 |      0.0 | 0.0 |       ... |       0.0 |       0.0 |       0.0 |       0.0 |      0.0 |         1.0 |         0.0 |         0.0 |         0.0 | 1.0 |
|    5 |    883 |   2 |  60 |        1 |     100.0 |     90.0 |     NaN |      0.0 |      0.0 | 0.0 |       ... |       0.0 |       0.0 |       0.0 |       0.0 |      0.0 |         0.0 |         0.0 |         0.0 |         0.0 | 0.0 |

In [ ]:
# apply one hot encoding to the variables in the categorical_variables list
# drop one of the generated columns to avoid collinearity



Let's see how many subjects we have in each category

In [ ]:
np.sum(cancer)

we don't have enough subjects in `ph.ecog_3.0` and `ph.ecog_nan`. We'll drop them.


In [ ]:
cancer = cancer.drop(columns=['ph.ecog_3.0', 'ph.ecog_nan'])

## Splitting the data in test / train

Before performing mean imputation and normalisation let's split the data in test and train (we should have done that before removing institutions with less than 10 subjects, but it would have complicated the script, so for sake of brevity we will pretend this didn't happen!).

Now split `cancer` in two dataframes: `train_data` and `test_data`, stratified on the columns `status` of the `cancer` dataframe.

In [ ]:
# Split the dataset in `train_data` and `test_data`, split by patient (patient-wise).

# your solution here

print(f"the train dataset has shape {train_data.shape}, the test dataset has shape {test_data.shape}")


## Missing values

Let's not impute missing values using mean imputation (different imputation strategies should be evaluated, probably using cross validation in the training set. Again, in the interest of time we will only look at mean imputation). 

Train a `sklearn.impute.SimpleImputer` using `mean` strategy on the training set, and transform both the training and the test sets.

Apply mean imputation to all the columns in the dataset.

In [ ]:
# apply mean imputation to remove all NaNs

# your solution here

train_data

## Normalisation

Let's now normalise each column. This will let us see the impact of each predictor in CoxPH models, and help with the fitting of the ML models.

Like before, train a `sklearn.preprocessing.Normalizer` on the trainig data only, and transform the training and test datasets. 

Apply normalisation to the columns in the `continuous_variables` list.

In [ ]:
# normalise the dataset




## Cox PH model

Let's now fit a CoxPH model.

In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(train_data, duration_col='time', event_col='status')

cph.print_summary()
cph.plot()

Let's see which of the covariates are significant:

In [ ]:
cox_covariates = cph.summary
cox_covariates[cox_covariates.p < 0.05]

We can see that `ph.karno` is a strong risk factor (high values increase ther risk), the same can be said for `ph.ecog`. Belonging to institution number 16 seems to be a strong protective factor.

Let's see how this model performs on the test set:

In [ ]:
cox_validation_score = cph.score(test_data, scoring_method="concordance_index")
print(f"CoxPH model has CI={cox_validation_score} in test, with CI={cph.concordance_index_} in training")

# we will append CI on test set for all models to this array
results = []

results.append(("CoxPH", cox_validation_score))

## CoxBoost

Let's now try using a boosting algorithm. We can use XGBoost or CoxBoost. For this exercise let's use CoxBoost, that is available in `scikit-survival`.

The models in `sksurv` expect `y` to be an array of tuples of the form `(Boolean, flaot)` where the first value is `True` if that subject experienced the event, `False` if censored, and the second value is the time to event (or to censoring).
In our dataset the event is present if `data.status == 2` where `data` can be `train_data` or `test_data`.


In [ ]:

def prepare_data_for_sksurv(data):
    X = data.loc[:, set(data.columns) - set(["time", "status"])]
    Y = pd.DataFrame({
        'event': data.status == 2,
        'time': data.time
        }).to_numpy()
    Y=np.array([tuple(row) for row in Y], dtype=[('event', '?'), ('time', '<f8')])
    return X, Y

X_train, y_train = prepare_data_for_sksurv(train_data)
X_test, y_test = prepare_data_for_sksurv(test_data)


Now use `sksurv.ensemble.GradientBoostingSurvivalAnalysis` to train and evaluate a CoxBoost model.

Use the `coxph` loss.

Perform a grid search for the parameters.

Use 5-fold-cross-validation on the trainng set to perform the hyperparameter search. 
Then train the best model on the whole training set, and evaluate its performance on the test set.

`sklearn.model_selection.GridSearchCV` will do all the heavy lifting for you!

The choice of which parameters to explore is up to you (be mindful that it might take a lot of time to perform a grid search over a large space). These are the paramters I used:

```
parameters = {
    'learning_rate':[0.001, 0.01, 0.05, 0.1, 0.25, 0.5],
    'n_estimators':[10, 50, 100, 250],
    'max_depth':[2,3,5]
        }
```

Append the CI on the test set to the `results` list like we did for the CoxPH model.

In [ ]:
# your solution here




## Survival Random Forests

Now repeat the same steps for Survival Random Forests, use `sksurv.ensemble.RandomSurvivalForest`. 
The code is pratically the same!

The choice of which parameters to explore is up to you (be mindful that it might take a lot of time to perform a grid search over a large space). These are the paramters I used:

```
parameters = {
    'min_samples_split':[2, 6, 12, 24, 48],
    'n_estimators':[10, 50, 100, 250],
    'max_depth':[2,4, 8, 16]
        }
```

Append the CI on the test set to the `results` list like we did for the other models.

In [ ]:
# your solution here



## Survival Suppoer Vector Machines

Let's now try Linear Survival SVM, use the `sksurv.svm.FastSurvivalSVM` class.
Set a value for `max_iter` higher than the default (I've set 100).
Like before use `GridSearchCV` to find the best value for `alpha`. 
I've set this for the parameters:
```
parameters = {'alpha': 2. ** np.arange(-4, 4, 1)}
```

Append the CI on the test set to the `results` list like we did for the other models.

In [ ]:
# your solution here


We have seen CoxPH, CoxBoost, Survival Random Forests and Survival SMVs. 
We have split the data in train test and used 5 fold cross validation to tune the hyperparameters.
In our task


In [ ]:
pd.DataFrame(results, columns = ["Model", "CI"]).sort_values("CI", ascending=False)

# Dynamic risk score

Now let's have a look at how to bring "time series" and "survival analysis" together.

## The dataset

Let's load the longitudinal dataset and explore the longitudinal and survival variables.

We use the `pbc2` dataset. This data is from the Mayo Clinic trial in primary biliary cirrhosis (PBC) of the liver conducted between 1974 and 1984. A total of 424 PBC patients, referred to Mayo Clinic during that ten-year interval met eligibility criteria for the randomized placebo controlled trial of the drug D-penicillamine, but only the first 312 cases in the data set participated in the randomized trial. Therefore, the data here are for the 312 patients with largely complete data.

See https://rdrr.io/cran/joineRML/man/pbc2.html

In [ ]:
pbc2 = pd.read_csv("data/pbc2.csv", index_col=0)

In [ ]:
pbc2.head()

In [ ]:
print(f"we have {len(pbc2.id.unique())} subjects, {len(pbc2)} records")

In [ ]:
# remove subjects with less than 3 entries
good_ids = pbc2.id.unique()[[len(pbc2.loc[pbc2.id == x, :])>=3 for x in pbc2.id.unique()]]
pbc2 = pbc2.loc[pbc2.id.isin(good_ids), :]

print(f"after removing subjects with less than 5 entries we have {len(pbc2.id.unique())} subjects, {len(pbc2)} records")

As we did before, we split the dataset in train and test. 

In [ ]:
from sklearn.model_selection import train_test_split

pbc2_survival = pbc2.loc[:, ["id", "status"]]
pbc2_survival = pbc2_survival.drop_duplicates()

train_ids, test_ids = train_test_split(
    pbc2_survival.id, 
    test_size = 0.2, 
    random_state = 42, 
    stratify=pbc2_survival.status)
train_data = pbc2.loc[pbc2.id.isin(train_ids), :]
test_data = pbc2.loc[pbc2.id.isin(test_ids), :]

# delete pbc2_survival, we'll re-instantiate later
del pbc2_survival
print(f"the train dataset has shape {train_data.shape}, the test dataset has shape {test_data.shape}")

## Longitudinal variables

In [ ]:

from sklearn.preprocessing import Normalizer

longitudinal_columns = list(train_data.columns[-9:-2])
columns_to_normalise = longitudinal_columns + ['age']

# like before, only use the training set to normalise
normalisation_means = np.nanmean(train_data.loc[:, columns_to_normalise], axis=0)
normalisation_sd = np.nanstd(train_data.loc[:, columns_to_normalise], axis=0)

print(f"""normalisation mean and std for columns {columns_to_normalise} are 
{normalisation_means} 
and 
{normalisation_sd}""")

normalised_columns = train_data.loc[:, columns_to_normalise]
normalised_columns = (normalised_columns - normalisation_means) / normalisation_sd
train_data.loc[:, columns_to_normalise] = normalised_columns

# also normalise the test set
test_data.loc[:, columns_to_normalise] = (test_data.loc[:, columns_to_normalise] - normalisation_means) / normalisation_sd


In [ ]:
n_limit = 50

def plot_longitudinal(longitudinal_var="serBilir"):
    for my_id in train_data.id.unique()[:n_limit]:
        my_data = train_data[train_data.id == my_id]
        plt.plot(my_data.year, my_data[longitudinal_var])

    plt.xlabel("years")
    plt.ylabel(longitudinal_var)
    plt.title(longitudinal_var)
    plt.show()

for x in longitudinal_columns:
    plot_longitudinal(x)


## Survival variables

In [ ]:
train_data_survival = train_data.loc[:, ["id", "years", "status", "drug", "age", "sex", "ascites", "hepatomegaly", "spiders", "edema"]]
train_data_survival.head()     

We have the following variables:
* `years`: the survival time
* `status`: the censoring status: our event is "dead", "alive" and "transplanted" are censored
* `drug`, `age`, `sex`, `ascitres`, `hepatomegaly`, `spiders`, `edema`: covariates, patient characteristic that don't change over time

`age` is continuous, the other covariates are either binary or categorical. Let's see which one are binary and which one will require 1 hot encoding.


In [ ]:
def print_values_of_variable(var_name):
    print(f"{var_name} has the following values {train_data_survival[var_name].unique()}")

vars = set(train_data_survival.columns[3:]) - {'age'}
for x in vars:
    print_values_of_variable(x)
    



* `edema` is categorical, with no NaNs. We will 1 hot encode this variable.
* `sex` and `drig` are binary, with no NaNs, we will turn these variables to 0s and 1s
* `spiders`, `hepatomegaly`, and `ascites` are binary with NaNs.  We will 1 hot encode this variable (so we don't drop subjects, being the dataset already small)
 


In [ ]:
to_be_1_hot_encoded = ['edema', 'spiders', 'hepatomegaly', 'ascites']
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')


In [ ]:
one_hot_encoder.fit(train_data_survival.loc[:,to_be_1_hot_encoded])


In [ ]:
train_data_survival_one_hot_encoded = pd.DataFrame(
    one_hot_encoder.transform(train_data_survival.loc[:,to_be_1_hot_encoded]).toarray(),
    columns=one_hot_encoder.get_feature_names(to_be_1_hot_encoded),
    index = train_data_survival.index)

train_data = pd.concat([train_data, train_data_survival_one_hot_encoded], axis=1)
train_data = train_data.drop(columns=to_be_1_hot_encoded)
train_data.drug = train_data.drug.apply(lambda x: 1 if x == "D-penicil" else 0)
train_data.sex = train_data.sex.apply(lambda x: 1 if x == "female" else 0)


In [ ]:
test_data_survival = test_data.loc[:, ["id", "years", "status", "drug", "age", "sex", "ascites", "hepatomegaly", "spiders", "edema"]]
test_data_survival_one_hot_encoded = pd.DataFrame(
    one_hot_encoder.transform(test_data_survival.loc[:,to_be_1_hot_encoded]).toarray(),
    columns=one_hot_encoder.get_feature_names(to_be_1_hot_encoded),
    index = test_data_survival.index)

test_data = pd.concat([test_data, test_data_survival_one_hot_encoded], axis=1)
test_data = test_data.drop(columns=to_be_1_hot_encoded)
test_data.drug = test_data.drug.apply(lambda x: 1 if x == "D-penicil" else 0)
test_data.sex = test_data.sex.apply(lambda x: 1 if x == "female" else 0)
        

## Prepare data for DL models

Now we'll prepare the data for tensorflow models.

We'll resample data to be equally sampled once a year.

We'll only implement the simplest topology possible, using for every time $T_i$ the last 5 years of data and try to predict survival probability for the following 5 yeaers.


The following code contains 1 *HUGE* bug, see if you can find it!



In [ ]:
from scipy import interpolate

continuous_columns = ['serBilir',
    'serChol', 'albumin', 'alkaline', 'SGOT', 'platelets', 'prothrombin']

categorical_columns = ['histologic', 'edema_No edema',
    'edema_edema despite diuretics', 'edema_edema no diuretics',
    'spiders_No', 'spiders_Yes', 'spiders_nan', 'hepatomegaly_No',
    'hepatomegaly_Yes', 'hepatomegaly_nan', 'ascites_No', 'ascites_Yes',
    'ascites_nan']

fixed_columns = ['id', 'years', 'status', 'drug', 'age', 'sex']

max_years = 6.0


def resample_to_yearly_grid(my_data):

    x_grid = np.arange(0, np.max(my_data.year), 1.0)

    my_target = "serBilir"

    def interpolate_continuous(my_target):
        f = interpolate.interp1d(my_data.year, my_data[my_target])
        return pd.DataFrame({my_target: f(x_grid)})

    def interpolate_categorical(my_target):
        f = interpolate.interp1d(my_data.year, my_data[my_target], kind="nearest")
        return pd.DataFrame({my_target: f(x_grid)})



    resampled_continuous = pd.concat([interpolate_continuous(x) for x in continuous_columns], axis=1)
    resampled_cat = pd.concat([interpolate_categorical(x) for x in categorical_columns], axis=1)
    my_resampled = pd.concat([resampled_continuous, resampled_cat], axis=1)
    for x in fixed_columns:
        my_resampled[x] = my_data[x].iloc[0]
    my_resampled["year"] = x_grid
    return my_resampled

def get_x_y_sets(dataset):
    def generate_data_for_id(my_id, dataset):
        my_data = dataset[dataset.id == my_id]

        my_data = resample_to_yearly_grid(my_data)

        y = my_data.year


        def generate_column_for_time(i):
            def my_f(x):
                if my_data.status.iloc[0] == "dead":
                    return x + i > my_data.years.iloc[0] 
                else:
                    return False
            return my_data.year.apply(my_f)
        def generate_mask_for_time(i):
            def my_f(x):
                if my_data.status.iloc[0] == "dead":
                    return 0
                elif x + i < my_data.years.iloc[0]:
                    return 0
                else:
                    return 1
            return my_data.year.apply(my_f)
        y = pd.DataFrame([generate_column_for_time(i) for i in np.arange(1.0, max_years, 1.0)]).transpose()
        y.columns = [f"year_{int(x)}" for x in np.arange(1.0, max_years, 1.0)]
        y_mask = pd.DataFrame([generate_mask_for_time(i) for i in np.arange(1.0, max_years, 1.0)]).transpose()
        y_mask.columns = [f"mask_year_{int(x)}" for x in np.arange(1.0, max_years, 1.0)]

        y = pd.concat([y, y_mask], axis=1)

        def generate_data_for_column(c):
            return pd.DataFrame([my_data.loc[:, c].shift(i, axis=0) for i in range(max(5, len(my_data)))]).transpose().iloc[:, 0:5]

        x = [generate_data_for_column(c) for c in longitudinal_columns] 
        x = pd.concat(x, axis=1)
        x = pd.concat([x, my_data.loc[:, set(my_data.columns) - set(longitudinal_columns)]], axis=1)
        x_mask = x.loc[:, continuous_columns].isna()
        x = x.fillna(0)
        x = pd.concat([x, x_mask], axis=1)
        x["age"] = x.age + x.year
        x = x.drop(columns=["status", "year"])
        return x.astype("float"), y.astype("float")

    data = [generate_data_for_id(x, dataset) for x in dataset.id.unique()]

    x, y = zip(*data)
    x = pd.concat(x)
    y = pd.concat(y)
    return x, y

x_train, y_train = get_x_y_sets(train_data)
print(f"Train: x has shape {x_train.shape}, y has shape {y_train.shape}")

x_test, y_test = get_x_y_sets(test_data)
print(f"Test: x has shape {x_test.shape}, y has shape {y_test.shape}")


In [ ]:
print("removing columns with zero variance")
x_test = x_test.loc[:, (x_train != x_train.iloc[0]).any()] 
x_train = x_train.loc[:, (x_train != x_train.iloc[0]).any()] 

train_ids = x_train.id
x_train = x_train.drop(columns=["id"])
test_ids = x_test.id
x_test = x_test.drop(columns=["id"])

print(f"Train: x has shape {x_train.shape}, y has shape {y_train.shape}")
print(f"Test: x has shape {x_test.shape}, y has shape {y_test.shape}")


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def censored_binary_crossentropy(y_true, y_pred):
    l = int(y_true.shape[1]/2)
    y_true_mask = y_true[:,l:]
    y_true_val = y_true[:,:l]
    _epsilon = tf.keras.backend.epsilon()
    y_pred_clipped = tf.clip_by_value(y_pred, _epsilon, 1 - _epsilon)
    cross_entropy = - y_true_val * tf.math.log(y_pred_clipped) - (1.0 - y_true_val) * tf.math.log(1.0 - y_pred_clipped)
    masked_cross_entropy = tf.where(y_true_mask == 1, tf.zeros_like(y_true_val), cross_entropy)
    return tf.reduce_mean(masked_cross_entropy)


In [ ]:
inputs = keras.Input(shape=(x_train.shape[1]))
x = layers.Dense(16, activation="relu")(inputs)
x = layers.Dense(int(y_train.shape[1]/2), activation="sigmoid")(x)
model = keras.Model(inputs, x)

epochs = 300

callbacks = [
    keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    )
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3, clipvalue=0.5),
    loss = censored_binary_crossentropy,
)

print(model.summary())

history = model.fit(
    x_train, y_train, 
    epochs=epochs, 
    callbacks=callbacks, 
    validation_data=(x_test, y_test),
    verbose=0
)

print(f"model achieved {model.evaluate(x_test, y_test)} accuracy in test")

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.show()


In [ ]:
import sklearn

In [ ]:
my_preds = pd.DataFrame(model.predict(x_test))
my_preds = pd.concat([my_preds, y_test.reset_index(drop=True)], axis=1)

def calc_scores(i):
    y_true = y_test.iloc[:, i]
    y_mask = y_test.iloc[:, i + 5]
    y_pred = my_preds.iloc[:, i]
    F1 = sklearn.metrics.f1_score(y_true, np.round(y_pred), sample_weight = 1.0 - y_mask)
    precision = sklearn.metrics.precision_score(y_true, np.round(y_pred), sample_weight = 1.0 - y_mask)
    recall = sklearn.metrics.recall_score(y_true, np.round(y_pred), sample_weight = 1.0 - y_mask)
    return precision, recall, F1, np.sum(y_mask)/len(y_mask)

scores = pd.DataFrame([calc_scores(i) for i in range(5)])
scores.columns = ["precision", "recall", "F1", "perc masked"]
scores.index = [f"at {i+1} year(s)" for i in range(5)]

plt.plot(scores.precision)
plt.plot(scores.recall)
plt.plot(scores.F1)

scores


If you didn't spot the bug you'll have very high F1 scores in test! 

HINT: If you haven't found the bug: study the train dataset... is there any column that is part of $x$ that is leaking the label?

After fixing the bug, what F1 scores do we get now?

Now try changing the network topology, activation functions, number fo layers, etc. Split the train data in train and validation, using 5-fold CV. 


In [ ]:
# Your solution here

Optional: add a second loss that approximates Concordance Index, or that predicts the values of longitudinal features at the next year

In [ ]:
# Your solution here